# Fine-tuning a LLM on your own docs

## Natural Language Processing

**Text classification**: the model is trained to predict a label for a given text. Text classification is frequently used for tasks like sentiment analysis, topic classification, and spam detection.

**Token classification**: the model is trained to predict a label for each token in the sequence. Token classification is frequently used for tasks like named entity recognition (NER), part-of-speech tagging, and chunking.

**Question answering**: the model is trained to predict an answer to a question based on a given context. Question answering is frequently used for tasks like question answering, fact verification, and conversational response generation.

**Causal language modeling**: the model is trained to predict the next token in the sequence. Causal language models are frequently used for text generation.

**Masked language modeling**: the model is trained to predict the masked tokens in the sequence. Masked language model can attend to tokens bidirectionally. This means the model has full access to the tokens on the left and right. Masked language modeling is great for tasks that require a good contextual understanding of an entire sequence. BERT is an example of a masked language model.

**Translation**: the model is trained to translate text from one language to another. Translation is frequently used for tasks like machine translation.

**Summarization**: the model is trained to summarize a given text. Summarization is frequently used for tasks like news summarization, article summarization, and book summarization.

**Multiple choice**: the model is trained to predict the correct answer from a list of multiple choice options. 

The goal of this post is to fine-tune a LLM on a custom dataset for causal language modeling. We will use the DistilGPT2 model. DistilGPT2 (short for Distilled-GPT2) is an English-language model pre-trained with the supervision of the smallest version of Generative Pre-trained Transformer 2 (GPT-2). Like GPT-2, DistilGPT2 can be used to generate text. Users of this model card should also consider information about the design, training, and limitations of GPT-2.

## Hugging Face Datasets

[Hugging Face Datasets](https://huggingface.co/docs/datasets/en/index)

Datasets is a library for easily accessing and sharing datasets for Audio, Computer Vision, and Natural Language Processing (NLP) tasks.
Backed by the Apache Arrow format, process large datasets with zero-copy reads without any memory constraints for optimal speed and efficiency.

Apache Arrow is a universal columnar format and multi-language toolbox for fast data interchange and in-memory analytics. What is a columnar format?

A columnar format stores data in columns, which are arrays of data. Each column contains data of the same type. Columnar formats are efficient for processing large datasets because they allow for efficient data access and processing.

![Columnar format](../assets/images/apache-arrow-columnar-representation.png)

RDBMS stores data row-wise, which means that each row is stored sequentially in memory. By contrast, columnar formats store data column-wise, which means that each column is stored sequentially in memory. While RDBMs is suited for OLTP (Online Transaction Processing), banking, retail, columnar formats are suited for OLAP (Online Analytical Processing), analytics, data warehousing (Hadoop, Spark).

Apache Parquet is a columnar storage format that is widely used in the Hadoop ecosystem. What is the difference then between Parquet and Arrow? Arrow is a memory format, while Parquet is a file format. Arrow is a multi-language toolbox, while Parquet is a storage format. Parquet data cannot be directly operated on, instead it must be first decoded in chunks.


Apache Arrow contains a set of technologies that enable data systems to efficiently store, process, and move data:

- **Arrow IPC/Serialization**: a binary format for fast and efficient data interchange and in-memory analytics.
- **Arrow Flight and Flight SQL**: High volume transport protocol and SQL-on-Arrow for scalable in-memory analytics.
- **Arrow Memory Pool**: a memory allocator for efficient memory management and garbage collection.

## Peft

[Peft](https://github.com/huggingface/peft) is a library that allows you to use LoRA (Low-Rank Adaptation) to fine-tune LLMs.

## Install libraries


In [ ]:
%pip install -U transformers torch tqdm tiktoken markdown

## Load the data

Let's load the ELI5 dataset from the Hugging Face Hub first. Next, we split the dataset into a training and test set, train the model on the training set, evaluate the model on the test set, inference the model on a custom prompt.

Next, we fine-tune the model on our own dataset.

In [1]:
from datasets import load_dataset

eli5 = load_dataset("eli5_category", split="train[:5000]")

eli5 = eli5.train_test_split(test_size=0.2)

/Users/LSoica/work/AI/blog/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 5411/5411 [00:00<00:00, 29139.00 examples/s]


## Preprocess

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")


eli5 = eli5.flatten()
eli5["train"][0]

{'q_id': '7h2ns1',
 'title': 'How does different data traveling on the same cable not get lost with all the other data?',
 'selftext': '',
 'category': 'Technology',
 'subreddit': 'explainlikeimfive',
 'answers.a_id': ['dqnmb02'],
 'answers.text': ['Data is encapsulated into packets with headers and trailers that identify it. Sometimes it does get lost though. When 2 devices establish a connection, they decide on packet numbering. If I send you a packet that says it contains data 1500 - 1600 you expect that my next packet starts with 1700. If it doesn’t, then you your response to me is essentially “I need 1700”.'],
 'answers.score': [10],
 'answers.text_urls': [[]],
 'title_urls': ['url'],
 'selftext_urls': ['url']}

In [3]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]])

tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=eli5["train"].column_names,
)

block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]Token indices sequence length is longer than the specified maximum sequence length for this model (1209 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1186 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2070 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1601 > 1024). Running this sequence through the model will result in indexing errors
Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]Token indices sequence length is longer than the specified maximum sequence length for this model (1246 > 1024). Running this sequence

## Train

In [4]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")

training_args = TrainingArguments(
    output_dir="my_awesome_eli5_clm-model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

/var/folders/59/c32_bthx48jd9m2ym5m3tnpw0000j7/T/ipykernel_46283/2004268791.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
 13%|█▎        | 500/3975 [04:26<30:20,  1.91it/s]

{'loss': 3.9787, 'grad_norm': 4.818574905395508, 'learning_rate': 1.748427672955975e-05, 'epoch': 0.38}


 25%|██▌       | 1000/3975 [08:52<25:56,  1.91it/s] 

{'loss': 3.9528, 'grad_norm': 3.8537325859069824, 'learning_rate': 1.4968553459119497e-05, 'epoch': 0.75}


                                                   
 33%|███▎      | 1325/3975 [12:26<23:20,  1.89it/s]

...

 88%|████████▊ | 3500/3975 [32:22<04:15,  1.86it/s]

{'loss': 3.8126, 'grad_norm': 4.447389125823975, 'learning_rate': 2.389937106918239e-06, 'epoch': 2.64}


                                                   
100%|██████████| 3975/3975 [37:30<00:00,  1.77it/s]

{'eval_loss': 3.8222262859344482, 'eval_runtime': 52.5303, 'eval_samples_per_second': 47.249, 'eval_steps_per_second': 5.92, 'epoch': 3.0}
{'train_runtime': 2250.3542, 'train_samples_per_second': 14.131, 'train_steps_per_second': 1.766, 'train_loss': 3.8763028203616354, 'epoch': 3.0}


TrainOutput(global_step=3975, training_loss=3.8763028203616354, metrics={'train_runtime': 2250.3542, 'train_samples_per_second': 14.131, 'train_steps_per_second': 1.766, 'total_flos': 1038654583603200.0, 'train_loss': 3.8763028203616354, 'epoch': 3.0})

## Evaluate

In [5]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

100%|██████████| 311/311 [00:40<00:00,  7.62it/s]

Perplexity: 45.71


## Inference


In [8]:
prompt = "Somatic hypermutation allows the immune system to"

from transformers import pipeline

generator = pipeline("text-generation", model="my_awesome_eli5_clm-model")
generator(prompt)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'generated_text': 'Somatic hypermutation allows the immune system to adapt to a more drastic response. When the immune system is overwhelmed, the immune system does NOT adapt to new stimuli (an event) and does not respond to new stimuli.This technique is called hyp'}]

## Build our own dataset


In [3]:
from datasets import load_dataset
own_dataset = load_dataset("text", data_files={"train": ["llm-fine-tuning-docs/docs/README.md"]}, split="train")


In [4]:
own_dataset = own_dataset.train_test_split(test_size=0.2)

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")

def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["text"]])

tokenized_own_dataset = own_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=own_dataset["train"].column_names,
)

block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_dataset = tokenized_own_dataset.map(group_texts, batched=True, num_proc=4)



Map (num_proc=4):  37%|███▋      | 41000/110255 [00:00<00:01, 47593.35 examples/s]Token indices sequence length is longer than the specified maximum sequence length for this model (2635 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1434 > 1024). Running this sequence through the model will result in indexing errors
Map (num_proc=4): 100%|██████████| 27564/27564 [00:01<00:00, 16234.17 examples/s]


In [4]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [5]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")

training_args = TrainingArguments(
    output_dir="my_awesome_eli5_clm-model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

/var/folders/59/c32_bthx48jd9m2ym5m3tnpw0000j7/T/ipykernel_2713/2004268791.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  2%|▏         | 500/25233 [04:33<3:39:04,  1.88it/s]

{'loss': 1.0723, 'grad_norm': 3.0989766120910645, 'learning_rate': 1.9603693575872867e-05, 'epoch': 0.06}


  4%|▍         | 1000/25233 [09:04<3:37:09,  1.86it/s]

{'loss': 0.8407, 'grad_norm': 2.9212241172790527, 'learning_rate': 1.9207387151745732e-05, 'epoch': 0.12}


  6%|▌         | 1500/25233 [13:36<3:35:21,  1.84it/s]

...

 99%|█████████▉| 25000/25233 [3:57:55<02:05,  1.86it/s]

{'loss': 0.5089, 'grad_norm': 2.8494760990142822, 'learning_rate': 1.84678793643245e-07, 'epoch': 2.97}


                                                       
100%|██████████| 25233/25233 [4:04:46<00:00,  1.72it/s]

{'eval_loss': 0.4647159278392792, 'eval_runtime': 277.1876, 'eval_samples_per_second': 60.349, 'eval_steps_per_second': 7.544, 'epoch': 3.0}
{'train_runtime': 14686.5169, 'train_samples_per_second': 13.744, 'train_steps_per_second': 1.718, 'train_loss': 0.5803486357610933, 'epoch': 3.0}


TrainOutput(global_step=25233, training_loss=0.5803486357610933, metrics={'train_runtime': 14686.5169, 'train_samples_per_second': 13.744, 'train_steps_per_second': 1.718, 'total_flos': 6592908962562048.0, 'train_loss': 0.5803486357610933, 'epoch': 3.0})

In [7]:
prompt = "Somatic hypermutation allows the immune system to"

from transformers import pipeline

generator = pipeline("text-generation", model="my_awesome_eli5_clm-model")
generator(prompt)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'generated_text': 'Somatic hypermutation allows the immune system to t i m e   |                                 '}]

## Now let's fine-tune the model with QLoRA

- target_modules must only be defined if the model arch is not supported by default by PEFT. (see [PEFT code](https://github.com/huggingface/peft/blob/39ef2546d5d9b8f5f8a7016ec10657887a867041/src/peft/utils/other.py#L220C1-L220C51))

In [43]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2").to("mps")

print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


In [44]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")

def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["text"]])

tokenized_own_dataset = own_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=own_dataset["train"].column_names,
)

block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_dataset = tokenized_own_dataset.map(group_texts, batched=True, num_proc=4)

from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [45]:

batch_size = 32

training_args = TrainingArguments(
    output_dir="my_awesome_eli5_clm-model",
    eval_strategy="epoch",
    learning_rate=1e-5,
    weight_decay=0.0001,
    push_to_hub=False,
    per_device_train_batch_size=batch_size,
)

peft_config = LoraConfig(
        r=64,
        lora_alpha=128,
        lora_dropout=0.1,
        # bias="none",
        # task_type="CAUSAL_LM",
        # target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

print(f"Trainable parameters for the base model: {model.num_parameters(only_trainable=True)}")

model = get_peft_model(model, peft_config)

print(f"Trainable parameters for the LoRA model: {model.num_parameters(only_trainable=True)}")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
    # tokenizer=tokenizer,
)

trainer.train()

Trainable parameters for the base model: 81912576
Trainable parameters for the LoRA model: 1179648


  8%|▊         | 500/6309 [13:36<2:34:54,  1.60s/it]

{'loss': 1.4383, 'grad_norm': 0.24702320992946625, 'learning_rate': 9.207481375812333e-06, 'epoch': 0.24}


 16%|█▌        | 1000/6309 [26:56<2:20:18,  1.59s/it]

{'loss': 1.2987, 'grad_norm': 0.4560306668281555, 'learning_rate': 8.414962751624663e-06, 'epoch': 0.48}


...

 95%|█████████▌| 6000/6309 [2:44:55<07:38,  1.48s/it]

{'loss': 1.0928, 'grad_norm': 0.6788768172264099, 'learning_rate': 4.897765097479791e-07, 'epoch': 2.85}


100%|██████████| 6309/6309 [2:52:49<00:00,  5.29s/it]




















































































































































































































































































































































































































































































































































































































































































































































































































































































































































































{'eval_runtime': 286.995, 'eval_samples_per_second': 58.287, 'eval_steps_per_second': 7.286, 'epoch': 3.0}
{'train_runtime': 10657.7698, 'train_samples_per_second': 18.939, 'train_steps_per_second': 0.592, 'train_loss': 1.17203244438903, 'epoch': 3.0}


TrainOutput(global_step=6309, training_loss=1.17203244438903, metrics={'train_runtime': 10657.7698, 'train_samples_per_second': 18.939, 'train_steps_per_second': 0.592, 'total_flos': 6775780751179776.0, 'train_loss': 1.17203244438903, 'epoch': 3.0})

## Infer

In [46]:
prompt = "Somatic hypermutation allows the immune system to"

from transformers import pipeline

generator = pipeline("text-generation", model="my_awesome_eli5_clm-model")
generator(prompt)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'generated_text': 'Somatic hypermutation allows the immune system to r a n g   n a m e   t o   s u p p o r t / h e a d l e r s   i n   S c'}]

## Now let's fine-tune Qwen/Qwen2.5-0.5B model with QLoRA

- target_modules must only be defined if the model arch is not supported by default by PEFT. (see [PEFT code](https://github.com/huggingface/peft/blob/39ef2546d5d9b8f5f8a7016ec10657887a867041/src/peft/utils/other.py#L220C1-L220C51))

In [6]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B").to("mps")

print(model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B")

def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["text"]])

tokenized_own_dataset = own_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=own_dataset["train"].column_names,
)

block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_dataset = tokenized_own_dataset.map(group_texts, batched=True, num_proc=4)

from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Map (num_proc=4): 100%|██████████| 27564/27564 [00:00<00:00, 41738.77 examples/s]


In [8]:

batch_size = 8

training_args = TrainingArguments(
    output_dir="my_awesome_eli5_clm-model",
    eval_strategy="epoch",
    learning_rate=1e-5,
    weight_decay=0.0001,
    push_to_hub=False,
    per_device_train_batch_size=batch_size,
)

peft_config = LoraConfig(
        r=64,
        lora_alpha=128,
        lora_dropout=0.1,
        # bias="none",
        # task_type="CAUSAL_LM",
        # target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

print(f"Trainable parameters for the base model: {model.num_parameters(only_trainable=True)}")

model = get_peft_model(model, peft_config)

print(f"Trainable parameters for the LoRA model: {model.num_parameters(only_trainable=True)}")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
    # tokenizer=tokenizer,
)

trainer.train()

Trainable parameters for the base model: 494032768
'NoneType' object has no attribute 'cadam32bit_grad_fp32'
Trainable parameters for the LoRA model: 4325376


/Users/LSoica/work/AI/blog/.venv/lib/python3.12/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
  5%|▌         | 500/9615 [39:28<4:37:32,  1.83s/it]   

{'loss': 2.0717, 'grad_norm': 3.0585858821868896, 'learning_rate': 9.479979199167968e-06, 'epoch': 0.16}


 10%|█         | 1000/9615 [3:29:29<24:59:46, 10.45s/it] 

{'loss': 1.766, 'grad_norm': 3.2948787212371826, 'learning_rate': 8.959958398335933e-06, 'epoch': 0.31}


...

                                                      
100%|██████████| 9615/9615 [13:23:21<00:00,  4.13s/it]

{'eval_runtime': 452.6747, 'eval_samples_per_second': 14.076, 'eval_steps_per_second': 1.761, 'epoch': 3.0}


100%|██████████| 9615/9615 [13:23:28<00:00,  5.01s/it]

{'train_runtime': 48208.209, 'train_samples_per_second': 1.595, 'train_steps_per_second': 0.199, 'train_loss': 1.507183917239502, 'epoch': 3.0}


TrainOutput(global_step=9615, training_loss=1.507183917239502, metrics={'train_runtime': 48208.209, 'train_samples_per_second': 1.595, 'train_steps_per_second': 0.199, 'total_flos': 2.1396524049432576e+16, 'train_loss': 1.507183917239502, 'epoch': 3.0})

## Infer

In [13]:
prompt = "What is a LLM?"
messages = [
    {"role": "system", "content": "You are an AI assistant created by Indigo."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


A LLM is a program that can be used to generate human readable text based on a sequence of input tokens. It is a type of natural language processing (NLP) algorithm that can be used to generate human readable text based on a sequence of input tokens. LLMs are commonly used for a variety of tasks, such as text generation, question answering, and information retrieval. They are often used in natural language processing applications where a human user needs to be able to understand and interpret the generated text. LLMs can be trained on large amounts of text data to improve their performance and accuracy. Some examples of LLMs include GPT-3, BERT, and GPT-2.
